# Projet : Détection de Fake News via Pipeline RAG
#### Étape 1 — Prétraitement des articles
##### Objectif de ce notebook :
##### Nettoyer et préparer les articles pour la recherche et la génération.
##### On effectue ici les traitements de texte nécessaires avant la vectorisation
##### et le stockage dans la base de données vectorielle (ChromaDB).


In [14]:
# 1. Importations et configuration initiale
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import warnings
import spacy
from tqdm import tqdm

# Ignorer certains warnings inutiles
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Charger le modèle spaCy pour la lemmatisation
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    import subprocess, sys
    subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")

# Définir les chemins de données
RAW_PATH = "data/raw"
PROCESSED_PATH = "data/processed"
os.makedirs(RAW_PATH, exist_ok=True)
os.makedirs(PROCESSED_PATH, exist_ok=True)

#### 🎯 Objectif
L’objectif de cette étape est de transformer les fichiers CSV d’articles (`True.csv`, `Fake.csv`)
en un dataset nettoyé, structuré et prêt à être vectorisé.

#### Étapes du pipeline :
1. Chargement et fusion des jeux de données (Fake / True)
2. Vérifications de base (valeurs manquantes, doublons)
3. Nettoyage des textes :
   - Suppression des balises HTML
   - Suppression des URLs
   - Suppression des caractères spéciaux
   - Normalisation des espaces
   - Mise en minuscules
4. Suppression des stopwords et lemmatisation (avec spaCy)
5. Découpage des articles en *chunks* (morceaux)
6. Sauvegarde des données nettoyées


In [15]:
#3. Chargement des fichiers CSV
csv_files = [f for f in os.listdir(RAW_PATH) if f.endswith(".csv")]
print(f"Fichiers trouvés : {csv_files}")

dfs = []
for file_name in csv_files:
    df_tmp = pd.read_csv(os.path.join(RAW_PATH, file_name))
    # Ajouter une colonne label en fonction du nom du fichier
    df_tmp["label"] = "Fake" if "fake" in file_name.lower() else "True"
    dfs.append(df_tmp)

# Fusionner les DataFrames
df = pd.concat(dfs, ignore_index=True)
print(f"📊 Taille totale du dataset : {df.shape}")
df.head()


Fichiers trouvés : ['True.csv', 'Fake.csv']
📊 Taille totale du dataset : (44898, 5)


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",True


In [16]:
#4. Vérification des données
print("Valeurs manquantes par colonne :")
print(df.isna().sum())

print("\nNombre de doublons :", df.duplicated().sum())

# Suppression des doublons et des valeurs manquantes
df = df.drop_duplicates().fillna("")


Valeurs manquantes par colonne :
title      0
text       0
subject    0
date       0
label      0
dtype: int64

Nombre de doublons : 209


In [17]:
#5. Nettoyage des textes
def clean_text(text):
    """
    Nettoie un texte :
    - Supprime les balises HTML
    - Supprime les URLs
    - Supprime les caractères spéciaux
    - Supprime les espaces multiples
    - Convertit en minuscules
    """
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^A-Za-z0-9À-ÿ\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

df["clean_text"] = df["text"].apply(clean_text)

# Vérification du nettoyage
df[["text", "clean_text"]].head(3)


,text,clean_text
0,WASHINGTON (Reuters) - The head of a conservat...,washington reuters the head of a conservative ...
1,WASHINGTON (Reuters) - Transgender people will...,washington reuters transgender people will be ...
2,WASHINGTON (Reuters) - The special counsel inv...,washington reuters the special counsel investi...


In [18]:
#6. Suppression des stopwords et lemmatisation
from spacy.lang.en.stop_words import STOP_WORDS

def lemmatize_text(text):
    """Applique la lemmatisation et retire les stopwords"""
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text.lower() not in STOP_WORDS]
    return " ".join(tokens)

# Pour accélérer le test, on peut le faire sur un échantillon d'abord :
sample = df["clean_text"].iloc[:3].apply(lemmatize_text)
print(sample)

0    washington reuters head conservative republica...
1    washington reuters transgender people allow ti...
2    washington reuters special counsel investigati...
Name: clean_text, dtype: object


In [19]:
# 7. Découpage en chunks
def chunk_text(text, chunk_size=200, overlap=50):
    """
    Découpe un texte en morceaux de longueur fixe (en mots)
    avec un chevauchement pour conserver le contexte.
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        if chunk:
            chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# Exemple
example_text = df["clean_text"].iloc[0]
print(chunk_text(example_text, chunk_size=50, overlap=10)[:2])


['washington reuters the head of a conservative republican faction in the u s congress who voted this month for a huge expansion of the national debt to pay for tax cuts called himself a fiscal conservative on sunday and urged budget restraint in 2018 in keeping with a sharp pivot', 'budget restraint in 2018 in keeping with a sharp pivot under way among republicans u s representative mark meadows speaking on cbs face the nation drew a hard line on federal spending which lawmakers are bracing to do battle over in january when they return from the holidays on wednesday']


In [20]:
#8. Application du découpage à tout le dataset
all_chunks = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    chunks = chunk_text(row["clean_text"], chunk_size=200, overlap=50)
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            "article_id": idx,
            "title": row.get("title", ""),
            "chunk_id": i,
            "chunk_text": chunk,
            "label": row["label"]
        })

df_chunks = pd.DataFrame(all_chunks)
print(f"✅ Nombre total de chunks : {df_chunks.shape[0]}")
df_chunks.head()


100%|██████████| 44689/44689 [00:04<00:00, 10289.66it/s]


✅ Nombre total de chunks : 145486


,article_id,title,chunk_id,chunk_text,label
0,0,"As U.S. budget fight looms, Republicans flip t...",0,washington reuters the head of a conservative ...,True
1,0,"As U.S. budget fight looms, Republicans flip t...",1,increases for non defense discretionary spendi...,True
2,0,"As U.S. budget fight looms, Republicans flip t...",2,responsibility democratic u s representative j...,True
3,0,"As U.S. budget fight looms, Republicans flip t...",3,mean food stamps housing assistance medicare a...,True
4,0,"As U.S. budget fight looms, Republicans flip t...",4,from deportation and provides them with work p...,True


In [21]:
#9. Sauvegarde des fichiers nettoyés 
# Sauvegarde du dataset nettoyé complet
cleaned_path = os.path.join(PROCESSED_PATH, "articles_cleaned.csv")
df.to_csv(cleaned_path, index=False)

# Sauvegarde du dataset chunké
chunks_path = os.path.join(PROCESSED_PATH, "articles_chunks.csv")
df_chunks.to_csv(chunks_path, index=False)

print(f" Données nettoyées sauvegardées : {cleaned_path}")
print(f" Chunks sauvegardés : {chunks_path}")


 Données nettoyées sauvegardées : data/processed/articles_cleaned.csv
 Chunks sauvegardés : data/processed/articles_chunks.csv


In [22]:
#10. Tests simples 
# Vérifier qu’il n’y a pas de texte vide
assert (df_chunks["chunk_text"].str.strip() != "").all(), " Des chunks vides détectés !"

# Vérifier la cohérence du label
assert set(df_chunks["label"].unique()) <= {"True", "Fake"}, "Label invalide détecté !"

print("Tous les tests basiques sont passés.")

Tous les tests basiques sont passés.


#### ✅ Étapes réalisées :
- Chargement et fusion des fichiers True/Fake
- Nettoyage complet des textes (HTML, URLs, caractères spéciaux, etc.)
- Mise en minuscules
- Lemmatisation et suppression des stopwords (optionnelle)
- Découpage en chunks pour la vectorisation
- Sauvegarde des jeux de données nettoyés et prêts pour ChromaDB

#### 🚀 Étape suivante :
👉 Passer à la **vectorisation et au stockage dans ChromaDB**
où chaque chunk sera converti en vecteur à l’aide du modèle d’embedding (Ollama).


### Étape 2 : Vectorisation et stockage dans ChromaDB
#### Objectif :
##### Charger les chunks nettoyés, créer une collection dans ChromaDB,
##### générer les embeddings via Ollama et stocker le tout avec les métadonnées.

In [23]:
# 1 Installation des bibliothèques
# #!pip install chromadb ollama --quiet
import chromadb
from chromadb.utils import embedding_functions

In [24]:
#2. Chargement du dataset chunké
CHUNKS_PATH = os.path.join(PROCESSED_PATH, "articles_chunks.csv")
df_chunks = pd.read_csv(CHUNKS_PATH)
print(f"{df_chunks.shape[0]} chunks chargés depuis {CHUNKS_PATH}")
df_chunks.head(2)

145486 chunks chargés depuis data/processed/articles_chunks.csv


,article_id,title,chunk_id,chunk_text,label
0,0,"As U.S. budget fight looms, Republicans flip t...",0,washington reuters the head of a conservative ...,True
1,0,"As U.S. budget fight looms, Republicans flip t...",1,increases for non defense discretionary spendi...,True


In [25]:
#3. Initialisation du client ChromaDB et création de la collection

# Création (ou récupération) d’un client local
client = chromadb.Client()

# Nom de la collection
collection_name = "fake_news_chunks"

# Supprimer la collection si elle existe déjà (pour test propre)
if collection_name in [c.name for c in client.list_collections()]:
    client.delete_collection(name=collection_name)

# Créer la collection
collection = client.create_collection(name=collection_name)
print(f" Collection '{collection_name}' créée avec succès.")

 Collection 'fake_news_chunks' créée avec succès.


In [26]:
# 4. Définir la fonction d’embedding avec Ollama
#rq : Assure  d’avoir lancé le serveur Ollama en parallèle : Terminal → ollama serve
embedding_fn = embedding_functions.OllamaEmbeddingFunction(
    model_name="all-minilm"  # modèle léger conseillé
)

In [27]:
# 5. Sélection d’un petit échantillon (pour test rapide)

# Pour éviter un traitement trop long au début
sample_df = df_chunks.sample(10, random_state=42).reset_index(drop=True)
sample_df.head(2)

,article_id,title,chunk_id,chunk_text,label
0,26204,WATCH: One Of Trump’s Biggest Supporters Is O...,3,supporting donald trump for president for too ...,Fake
1,22451,Joe Biden Is Downright Ruthless When Rich ‘A*...,0,joe biden was speaking at the skybridge altern...,Fake


In [28]:
# 6. Ajout des chunks dans ChromaDB
for idx, row in sample_df.iterrows():
    chunk_id = f"{row['article_id']}_{row['chunk_id']}"
    metadata = {
        "title": row.get("title", ""),
        "label": row["label"],
        "article_id": int(row["article_id"]),
    }

    collection.add(
        ids=[chunk_id],
        documents=[row["chunk_text"]],
        metadatas=[metadata],
        embeddings=embedding_fn([row["chunk_text"]])
    )

print(f" {len(sample_df)} chunks ajoutés à la collection '{collection_name}'.")


 10 chunks ajoutés à la collection 'fake_news_chunks'.


In [29]:
# 7. Vérification du stockage
results = collection.peek()
print(" Exemple de document stocké :")
results

 Exemple de document stocké :


{'ids': ['26204_3',
  '22451_0',
  '16967_3',
  '39130_2',
  '44364_0',
  '31827_11',
  '32417_2',
  '7251_1',
  '21995_0',
  '10910_0'],
 'embeddings': array([[ 0.03148066,  0.06609543, -0.00260025, ..., -0.01345606,
         -0.00472292,  0.00255966],
        [-0.04013475, -0.01899961,  0.02693708, ...,  0.00223051,
         -0.12071282,  0.03988767],
        [-0.00988505,  0.04822049,  0.02915613, ...,  0.01965448,
         -0.03194366,  0.02829264],
        ...,
        [ 0.04085801, -0.00852068,  0.0645113 , ..., -0.05173074,
          0.04013757, -0.03626084],
        [-0.01746017, -0.0362521 ,  0.10753163, ..., -0.0122874 ,
          0.00103011, -0.02215472],
        [-0.0513751 , -0.01095264,  0.01819525, ..., -0.09877898,
          0.01238176,  0.09949215]], shape=(10, 384)),
 'documents': ['supporting donald trump for president for too long african american votes have been taken for granted by democratic politicians and enough is enough so burns literally played the race card

In [30]:
# 8. Test rapide de recherche sémantique
query = "The president made a false statement about the election"
results = collection.query(query_texts=[query], n_results=3)

print(" Résultats de la recherche sémantique :")
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print(f"- [{meta['label']}] {doc[:150]}...\n")

 Résultats de la recherche sémantique :
- [Fake] s leaks to confabulate that russian s hacked the dnc to influence the elections is the claim of one well known russian spy then 17 u s intelligence ag...

- [Fake] supporting donald trump for president for too long african american votes have been taken for granted by democratic politicians and enough is enough s...

- [Fake] joe biden was speaking at the skybridge alternatives salt conference when one rich a hole decided to disrespect the former vice president s dead son b...



In [31]:
# 9. Sauvegarde et fin de l’étape 2
print("""
Étape 2 terminée : les chunks ont été vectorisés et stockés dans ChromaDB.

🎯 Prochaines étapes :
→ Développer le pipeline RAG :
   1. Récupération d’un article soumis par l’utilisateur
   2. Recherche sémantique des chunks similaires
   3. Envoi du contexte au modèle Ollama pour verdict (True/Fake) et justification
""")


Étape 2 terminée : les chunks ont été vectorisés et stockés dans ChromaDB.

🎯 Prochaines étapes :
→ Développer le pipeline RAG :
   1. Récupération d’un article soumis par l’utilisateur
   2. Recherche sémantique des chunks similaires
   3. Envoi du contexte au modèle Ollama pour verdict (True/Fake) et justification



#### Étape 3 — Système RAG : Recherche et génération de verdict
##### Objectif :
##### Prendre un texte soumis par l'utilisateur, rechercher les chunks les plus similaires
##### dans ChromaDB et générer un verdict TRUE/FAKE avec justification via Ollama.


In [ ]:
# ===============================
# Pipeline RAG optimisé ChromaDB + Ollama
# ===============================

# 1️⃣ Importations
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from chromadb import Client
from chromadb.config import Settings
from chromadb.utils import embedding_functions

# -------------------------------
# 2️⃣ Créer ou réutiliser le client ChromaDB
CHROMA_DB_PATH = "./chroma_db"

try:
    client  # réutiliser si existe déjà
except NameError:
    settings = Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=CHROMA_DB_PATH
    )
    client = Client(settings=settings)

# -------------------------------
# 3️⃣ Définir la fonction d'embedding Ollama
embedding_fn = embedding_functions.OllamaEmbeddingFunction(model_name="all-minilm")

# -------------------------------
# 4️⃣ Créer ou récupérer la collection
collection_name = "fake_news_chunks"
if collection_name in [c.name for c in client.list_collections()]:
    collection = client.get_collection(name=collection_name)
    print(f"Collection '{collection_name}' récupérée.")
else:
    collection = client.create_collection(
        name=collection_name,
        embedding_function=embedding_fn
    )
    print(f"Collection '{collection_name}' créée.")

# -------------------------------
# 5️⃣ Charger les chunks nettoyés
CHUNKS_PATH = "data/processed/articles_chunks.csv"
df_chunks = pd.read_csv(CHUNKS_PATH)
print(f"{df_chunks.shape[0]} chunks chargés depuis {CHUNKS_PATH}")

# -------------------------------
# # -------------------------------
# 6️⃣ Ajouter les chunks dans la collection (batch sécurisé)
existing_ids = set()
all_meta = collection.get(include=["metadatas"])["metadatas"]
for m in all_meta:
    if isinstance(m, list):
        for item in m:
            if isinstance(item, dict) and "id" in item:
                existing_ids.add(item["id"])

print(f"{len(existing_ids)} chunks déjà présents dans la collection.")

batch_size = 50  # batch plus petit pour éviter dépassement
max_chunk_length = 512  # tronquer chaque chunk à 512 caractères

for start in tqdm(range(0, len(df_chunks), batch_size), desc="Ajout chunks"):
    batch = df_chunks.iloc[start:start+batch_size]

    # Générer les ids
    ids = [f"{row['article_id']}_{row['chunk_id']}" for _, row in batch.iterrows()]
    # Filtrer les chunks déjà existants
    new_idx = [i for i, cid in enumerate(ids) if cid not in existing_ids]
    if not new_idx:
        continue

    batch = batch.iloc[new_idx]
    ids = [f"{row['article_id']}_{row['chunk_id']}" for _, row in batch.iterrows()]

    # Préparer metadatas
    metadatas = [
        {
            "title": row.get("title", ""),
            "label": row["label"],
            "article_id": int(row["article_id"]),
            "id": ids[i]
        }
        for i, (_, row) in enumerate(batch.iterrows())
    ]

    # Tronquer les chunks trop longs
    texts = [text[:max_chunk_length] for text in batch["chunk_text"].tolist()]

    # Calcul des embeddings avec gestion d’erreur
    try:
        embeddings = embedding_fn(texts)
    except Exception as e:
        print(f"⚠️ Erreur embedding pour ce batch ({start}-{start+batch_size}): {e}")
        continue

    # Ajout dans la collection
    collection.add(
        ids=ids,
        documents=texts,
        metadatas=metadatas,
        embeddings=embeddings
    )

print("✅ Tous les chunks ajoutés (ou ignorés si déjà présents).")


Collection 'fake_news_chunks' récupérée.
145486 chunks chargés depuis data/processed/articles_chunks.csv
0 chunks déjà présents dans la collection.


Ajout chunks: 100%|██████████| 2910/2910 [1:07:25<00:00,  1.39s/it]   


✅ Tous les chunks ajoutés (ou ignorés si déjà présents).
Exemple de document stocké : s...


ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got [[array([-1.81416776e-02, -1.66259743e-02,  2.08692756e-02, -1.75619125e-02,
        1.80299189e-02,  2.60097906e-02,  5.11432113e-03,  1.14210762e-01,
       -2.04584561e-02,  5.41193597e-02, -2.44015623e-02, -1.26532063e-01,
        7.31085055e-03, -1.38087375e-02, -8.85488391e-02, -5.60243055e-02,
       -4.70394045e-02,  1.15937643e-01, -1.96584035e-02,  2.52075624e-02,
        1.93452239e-02, -5.84553555e-02, -2.57454417e-03, -1.32696088e-02,
       -3.99976000e-02, -1.97132248e-02, -7.61035038e-03, -3.05432659e-02,
        1.28386142e-02, -3.06564849e-02,  9.19936523e-02,  2.72286125e-02,
        9.28371400e-02, -2.16248930e-02,  4.68153805e-02, -1.35617855e-03,
        3.60254794e-02, -4.95285392e-02,  1.55769046e-02,  5.89962769e-03,
       -7.28993416e-02, -2.07973160e-02, -7.61383250e-02, -5.29609285e-02,
        2.79479902e-02,  2.00565234e-02,  4.91

In [ ]:
# 7 # Fonction RAG pour verdict True/Fake avec Ollama
def rag_verdict_batch(user_queries):
    import ollama
    """
    Fonction qui prend une ou plusieurs déclarations (user_queries)
    et retourne un verdict True/False généré par le modèle Ollama.

    Étapes :
    1. Convertir la requête en liste si besoin.
    2. Initialiser une liste pour stocker les verdicts.
    3. Pour chaque déclaration :
    a) Créer un prompt pour le modèle.
    b) Générer la réponse via Ollama.
    c) Extraire uniquement le texte de la réponse, selon la structure.
    d) Nettoyer le texte pour ne récupérer que True/False.
    e) Gérer les exceptions et erreurs.
    """
# a/ S'assurer que l'entrée est une liste
    if isinstance(user_queries, str):
        user_queries = [user_queries]
# b/ Initialisation de la liste des verdicts
    verdicts = []
# c/  Boucle sur chaque déclaration utilisateur
    for query in user_queries:
        #) Création du prompt pour le modèle
        prompt = f"""
        You are a fact-checker AI.
        Check the following statement and answer only 'True' or 'False'.
        Statement: {query}
        """

        try:
            # Appel à l'API Ollama pour générer le verdict
            response = ollama.generate(model="phi3", prompt=prompt)
            
            # Extraire uniquement le texte de la réponse
            text = ""
            if hasattr(response, "response"):
                text = response.response.strip()
            elif hasattr(response, "message") and hasattr(response.message, "content"):
                text = response.message.content.strip()
            elif hasattr(response, "outputText"):
                text = response.outputText.strip()
            else:
                text = str(response).strip()

            # Nettoyer le texte pour récupérer seulement True/False
            if text.lower().startswith("true"):
                verdicts.append("True")
            elif text.lower().startswith("false"):
                verdicts.append("False")
            else:
                verdicts.append(text)  # Si réponse ambiguë, on garde le texte complet

        except Exception as e:
            #Gestion d'erreur : on ajoute "Erreur" au verdicts
            print(f"Erreur lors de la génération de la réponse : {e}")
            verdicts.append("Erreur")
    # Retourne la liste des verdicts pour toutes les déclarations
    return verdicts


In [ ]:
# Test rapide de la fonction
user_query_en = "The government announced a new economic measure to reduce unemployment."
verdict_en = rag_verdict_batch(user_query_en)
print("Verdict (English):", verdict_en[0])


Verdict (English): False


# Projet : Détection de Fake News via Pipeline RAG
#### Étape 1 — Prétraitement des articles
##### Objectif de ce notebook :
##### Nettoyer et préparer les articles pour la recherche et la génération.
##### On effectue ici les traitements de texte nécessaires avant la vectorisation
##### et le stockage dans la base de données vectorielle (ChromaDB).


In [ ]:
# 1. Importations et configuration initiale
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import warnings
import spacy
from tqdm import tqdm

# Ignorer certains warnings inutiles
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# Charger le modèle spaCy pour la lemmatisation
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    import subprocess, sys
    subprocess.run([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")

# Définir les chemins de données
RAW_PATH = "data/raw"
PROCESSED_PATH = "data/processed"
os.makedirs(RAW_PATH, exist_ok=True)
os.makedirs(PROCESSED_PATH, exist_ok=True)

#### 🎯 Objectif
L’objectif de cette étape est de transformer les fichiers CSV d’articles (`True.csv`, `Fake.csv`)
en un dataset nettoyé, structuré et prêt à être vectorisé.

#### Étapes du pipeline :
1. Chargement et fusion des jeux de données (Fake / True)
2. Vérifications de base (valeurs manquantes, doublons)
3. Nettoyage des textes :
   - Suppression des balises HTML
   - Suppression des URLs
   - Suppression des caractères spéciaux
   - Normalisation des espaces
   - Mise en minuscules
4. Suppression des stopwords et lemmatisation (avec spaCy)
5. Découpage des articles en *chunks* (morceaux)
6. Sauvegarde des données nettoyées


In [ ]:
#3. Chargement des fichiers CSV
csv_files = [f for f in os.listdir(RAW_PATH) if f.endswith(".csv")]
print(f"Fichiers trouvés : {csv_files}")

dfs = []
for file_name in csv_files:
    df_tmp = pd.read_csv(os.path.join(RAW_PATH, file_name))
    # Ajouter une colonne label en fonction du nom du fichier
    df_tmp["label"] = "Fake" if "fake" in file_name.lower() else "True"
    dfs.append(df_tmp)

# Fusionner les DataFrames
df = pd.concat(dfs, ignore_index=True)
print(f"📊 Taille totale du dataset : {df.shape}")
df.head()


Fichiers trouvés : ['True.csv', 'Fake.csv']
📊 Taille totale du dataset : (44898, 5)


,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",True
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",True
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",True
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",True
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",True


In [ ]:
#4. Vérification des données
print("Valeurs manquantes par colonne :")
print(df.isna().sum())

print("\nNombre de doublons :", df.duplicated().sum())

# Suppression des doublons et des valeurs manquantes
df = df.drop_duplicates().fillna("")


Valeurs manquantes par colonne :
title      0
text       0
subject    0
date       0
label      0
dtype: int64

Nombre de doublons : 209


In [ ]:
#5. Nettoyage des textes
def clean_text(text):
    """
    Nettoie un texte :
    - Supprime les balises HTML
    - Supprime les URLs
    - Supprime les caractères spéciaux
    - Supprime les espaces multiples
    - Convertit en minuscules
    """
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^A-Za-z0-9À-ÿ\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

df["clean_text"] = df["text"].apply(clean_text)

# Vérification du nettoyage
df[["text", "clean_text"]].head(3)


,text,clean_text
0,WASHINGTON (Reuters) - The head of a conservat...,washington reuters the head of a conservative ...
1,WASHINGTON (Reuters) - Transgender people will...,washington reuters transgender people will be ...
2,WASHINGTON (Reuters) - The special counsel inv...,washington reuters the special counsel investi...


In [ ]:
#6. Suppression des stopwords et lemmatisation
from spacy.lang.en.stop_words import STOP_WORDS

def lemmatize_text(text):
    """Applique la lemmatisation et retire les stopwords"""
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text.lower() not in STOP_WORDS]
    return " ".join(tokens)

# Pour accélérer le test, on peut le faire sur un échantillon d'abord :
sample = df["clean_text"].iloc[:3].apply(lemmatize_text)
print(sample)

0    washington reuters head conservative republica...
1    washington reuters transgender people allow ti...
2    washington reuters special counsel investigati...
Name: clean_text, dtype: object


In [ ]:
# 7. Découpage en chunks
def chunk_text(text, chunk_size=200, overlap=50):
    """
    Découpe un texte en morceaux de longueur fixe (en mots)
    avec un chevauchement pour conserver le contexte.
    """
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        if chunk:
            chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# Exemple
example_text = df["clean_text"].iloc[0]
print(chunk_text(example_text, chunk_size=50, overlap=10)[:2])


['washington reuters the head of a conservative republican faction in the u s congress who voted this month for a huge expansion of the national debt to pay for tax cuts called himself a fiscal conservative on sunday and urged budget restraint in 2018 in keeping with a sharp pivot', 'budget restraint in 2018 in keeping with a sharp pivot under way among republicans u s representative mark meadows speaking on cbs face the nation drew a hard line on federal spending which lawmakers are bracing to do battle over in january when they return from the holidays on wednesday']


In [ ]:
#8. Application du découpage à tout le dataset
all_chunks = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    chunks = chunk_text(row["clean_text"], chunk_size=200, overlap=50)
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            "article_id": idx,
            "title": row.get("title", ""),
            "chunk_id": i,
            "chunk_text": chunk,
            "label": row["label"]
        })

df_chunks = pd.DataFrame(all_chunks)
print(f"✅ Nombre total de chunks : {df_chunks.shape[0]}")
df_chunks.head()


100%|██████████| 44689/44689 [00:04<00:00, 10289.66it/s]


✅ Nombre total de chunks : 145486


,article_id,title,chunk_id,chunk_text,label
0,0,"As U.S. budget fight looms, Republicans flip t...",0,washington reuters the head of a conservative ...,True
1,0,"As U.S. budget fight looms, Republicans flip t...",1,increases for non defense discretionary spendi...,True
2,0,"As U.S. budget fight looms, Republicans flip t...",2,responsibility democratic u s representative j...,True
3,0,"As U.S. budget fight looms, Republicans flip t...",3,mean food stamps housing assistance medicare a...,True
4,0,"As U.S. budget fight looms, Republicans flip t...",4,from deportation and provides them with work p...,True


In [ ]:
#9. Sauvegarde des fichiers nettoyés 
# Sauvegarde du dataset nettoyé complet
cleaned_path = os.path.join(PROCESSED_PATH, "articles_cleaned.csv")
df.to_csv(cleaned_path, index=False)

# Sauvegarde du dataset chunké
chunks_path = os.path.join(PROCESSED_PATH, "articles_chunks.csv")
df_chunks.to_csv(chunks_path, index=False)

print(f" Données nettoyées sauvegardées : {cleaned_path}")
print(f" Chunks sauvegardés : {chunks_path}")


 Données nettoyées sauvegardées : data/processed/articles_cleaned.csv
 Chunks sauvegardés : data/processed/articles_chunks.csv


In [ ]:
#10. Tests simples 
# Vérifier qu’il n’y a pas de texte vide
assert (df_chunks["chunk_text"].str.strip() != "").all(), " Des chunks vides détectés !"

# Vérifier la cohérence du label
assert set(df_chunks["label"].unique()) <= {"True", "Fake"}, "Label invalide détecté !"

print("Tous les tests basiques sont passés.")

Tous les tests basiques sont passés.


#### ✅ Étapes réalisées :
- Chargement et fusion des fichiers True/Fake
- Nettoyage complet des textes (HTML, URLs, caractères spéciaux, etc.)
- Mise en minuscules
- Lemmatisation et suppression des stopwords (optionnelle)
- Découpage en chunks pour la vectorisation
- Sauvegarde des jeux de données nettoyés et prêts pour ChromaDB

#### 🚀 Étape suivante :
👉 Passer à la **vectorisation et au stockage dans ChromaDB**
où chaque chunk sera converti en vecteur à l’aide du modèle d’embedding (Ollama).


### Étape 2 : Vectorisation et stockage dans ChromaDB
#### Objectif :
##### Charger les chunks nettoyés, créer une collection dans ChromaDB,
##### générer les embeddings via Ollama et stocker le tout avec les métadonnées.

In [ ]:
# 1 Installation des bibliothèques
# #!pip install chromadb ollama --quiet
import chromadb
from chromadb.utils import embedding_functions

In [ ]:
#2. Chargement du dataset chunké
CHUNKS_PATH = os.path.join(PROCESSED_PATH, "articles_chunks.csv")
df_chunks = pd.read_csv(CHUNKS_PATH)
print(f"{df_chunks.shape[0]} chunks chargés depuis {CHUNKS_PATH}")
df_chunks.head(2)

145486 chunks chargés depuis data/processed/articles_chunks.csv


,article_id,title,chunk_id,chunk_text,label
0,0,"As U.S. budget fight looms, Republicans flip t...",0,washington reuters the head of a conservative ...,True
1,0,"As U.S. budget fight looms, Republicans flip t...",1,increases for non defense discretionary spendi...,True


In [ ]:
#3. Initialisation du client ChromaDB et création de la collection

# Création (ou récupération) d’un client local
client = chromadb.Client()

# Nom de la collection
collection_name = "fake_news_chunks"

# Supprimer la collection si elle existe déjà (pour test propre)
if collection_name in [c.name for c in client.list_collections()]:
    client.delete_collection(name=collection_name)

# Créer la collection
collection = client.create_collection(name=collection_name)
print(f" Collection '{collection_name}' créée avec succès.")

 Collection 'fake_news_chunks' créée avec succès.


In [ ]:
# 4. Définir la fonction d’embedding avec Ollama
#rq : Assure  d’avoir lancé le serveur Ollama en parallèle : Terminal → ollama serve
embedding_fn = embedding_functions.OllamaEmbeddingFunction(
    model_name="all-minilm"  # modèle léger conseillé
)

In [ ]:
# 5. Sélection d’un petit échantillon (pour test rapide)

# Pour éviter un traitement trop long au début
sample_df = df_chunks.sample(10, random_state=42).reset_index(drop=True)
sample_df.head(2)

,article_id,title,chunk_id,chunk_text,label
0,26204,WATCH: One Of Trump’s Biggest Supporters Is O...,3,supporting donald trump for president for too ...,Fake
1,22451,Joe Biden Is Downright Ruthless When Rich ‘A*...,0,joe biden was speaking at the skybridge altern...,Fake


In [ ]:
# 6. Ajout des chunks dans ChromaDB
for idx, row in sample_df.iterrows():
    chunk_id = f"{row['article_id']}_{row['chunk_id']}"
    metadata = {
        "title": row.get("title", ""),
        "label": row["label"],
        "article_id": int(row["article_id"]),
    }

    collection.add(
        ids=[chunk_id],
        documents=[row["chunk_text"]],
        metadatas=[metadata],
        embeddings=embedding_fn([row["chunk_text"]])
    )

print(f" {len(sample_df)} chunks ajoutés à la collection '{collection_name}'.")


 10 chunks ajoutés à la collection 'fake_news_chunks'.


In [ ]:
# 7. Vérification du stockage
results = collection.peek()
print(" Exemple de document stocké :")
results

 Exemple de document stocké :


{'ids': ['26204_3',
  '22451_0',
  '16967_3',
  '39130_2',
  '44364_0',
  '31827_11',
  '32417_2',
  '7251_1',
  '21995_0',
  '10910_0'],
 'embeddings': array([[ 0.03148066,  0.06609543, -0.00260025, ..., -0.01345606,
         -0.00472292,  0.00255966],
        [-0.04013475, -0.01899961,  0.02693708, ...,  0.00223051,
         -0.12071282,  0.03988767],
        [-0.00988505,  0.04822049,  0.02915613, ...,  0.01965448,
         -0.03194366,  0.02829264],
        ...,
        [ 0.04085801, -0.00852068,  0.0645113 , ..., -0.05173074,
          0.04013757, -0.03626084],
        [-0.01746017, -0.0362521 ,  0.10753163, ..., -0.0122874 ,
          0.00103011, -0.02215472],
        [-0.0513751 , -0.01095264,  0.01819525, ..., -0.09877898,
          0.01238176,  0.09949215]], shape=(10, 384)),
 'documents': ['supporting donald trump for president for too long african american votes have been taken for granted by democratic politicians and enough is enough so burns literally played the race card

In [ ]:
# 8. Test rapide de recherche sémantique
query = "The president made a false statement about the election"
results = collection.query(query_texts=[query], n_results=3)

print(" Résultats de la recherche sémantique :")
for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    print(f"- [{meta['label']}] {doc[:150]}...\n")

 Résultats de la recherche sémantique :
- [Fake] s leaks to confabulate that russian s hacked the dnc to influence the elections is the claim of one well known russian spy then 17 u s intelligence ag...

- [Fake] supporting donald trump for president for too long african american votes have been taken for granted by democratic politicians and enough is enough s...

- [Fake] joe biden was speaking at the skybridge alternatives salt conference when one rich a hole decided to disrespect the former vice president s dead son b...



In [ ]:
# 9. Sauvegarde et fin de l’étape 2
print("""
Étape 2 terminée : les chunks ont été vectorisés et stockés dans ChromaDB.

🎯 Prochaines étapes :
→ Développer le pipeline RAG :
   1. Récupération d’un article soumis par l’utilisateur
   2. Recherche sémantique des chunks similaires
   3. Envoi du contexte au modèle Ollama pour verdict (True/Fake) et justification
""")


Étape 2 terminée : les chunks ont été vectorisés et stockés dans ChromaDB.

🎯 Prochaines étapes :
→ Développer le pipeline RAG :
   1. Récupération d’un article soumis par l’utilisateur
   2. Recherche sémantique des chunks similaires
   3. Envoi du contexte au modèle Ollama pour verdict (True/Fake) et justification



#### Étape 3 — Système RAG : Recherche et génération de verdict
##### Objectif :
##### Prendre un texte soumis par l'utilisateur, rechercher les chunks les plus similaires
##### dans ChromaDB et générer un verdict TRUE/FAKE avec justification via Ollama.


In [ ]:
# ===============================
# Pipeline RAG optimisé ChromaDB + Ollama
# ===============================

# 1️ Importations
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from chromadb import Client
from chromadb.config import Settings
from chromadb.utils import embedding_functions

# -------------------------------
# 2️ Créer le client ChromaDB
CHROMA_DB_PATH = "./chroma_db"

try:
    client  # réutiliser si existe déjà
except NameError:
    settings = Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=CHROMA_DB_PATH
    )
    client = Client(settings=settings)

# -------------------------------
# 3️ Définir la fonction d'embedding Ollama
embedding_fn = embedding_functions.OllamaEmbeddingFunction(model_name="all-minilm")

# -------------------------------
# 4️ Créer ou récupérer la collection
collection_name = "fake_news_chunks"
if collection_name in [c.name for c in client.list_collections()]:
    collection = client.get_collection(name=collection_name)
    print(f"Collection '{collection_name}' récupérée.")
else:
    collection = client.create_collection(
        name=collection_name,
        embedding_function=embedding_fn
    )
    print(f"Collection '{collection_name}' créée.")

# -------------------------------
# 5️ Charger les chunks nettoyés
CHUNKS_PATH = "data/processed/articles_chunks.csv"
df_chunks = pd.read_csv(CHUNKS_PATH)
print(f"{df_chunks.shape[0]} chunks chargés depuis {CHUNKS_PATH}")

# -------------------------------
# # -------------------------------
# 6️ Ajouter les chunks dans la collection (batch sécurisé)
existing_ids = set()
all_meta = collection.get(include=["metadatas"])["metadatas"]
for m in all_meta:
    if isinstance(m, list):
        for item in m:
            if isinstance(item, dict) and "id" in item:
                existing_ids.add(item["id"])

print(f"{len(existing_ids)} chunks déjà présents dans la collection.")

batch_size = 50  # batch plus petit pour éviter dépassement
max_chunk_length = 512  # tronquer chaque chunk à 512 caractères

for start in tqdm(range(0, len(df_chunks), batch_size), desc="Ajout chunks"):
    batch = df_chunks.iloc[start:start+batch_size]

    # Générer les ids
    ids = [f"{row['article_id']}_{row['chunk_id']}" for _, row in batch.iterrows()]
    # Filtrer les chunks déjà existants
    new_idx = [i for i, cid in enumerate(ids) if cid not in existing_ids]
    if not new_idx:
        continue

    batch = batch.iloc[new_idx]
    ids = [f"{row['article_id']}_{row['chunk_id']}" for _, row in batch.iterrows()]

    # Préparer metadatas
    metadatas = [
        {
            "title": row.get("title", ""),
            "label": row["label"],
            "article_id": int(row["article_id"]),
            "id": ids[i]
        }
        for i, (_, row) in enumerate(batch.iterrows())
    ]

    # Tronquer les chunks trop longs
    texts = [text[:max_chunk_length] for text in batch["chunk_text"].tolist()]

    # Calcul des embeddings avec gestion d’erreur
    try:
        embeddings = embedding_fn(texts)
    except Exception as e:
        print(f"Erreur embedding pour ce batch ({start}-{start+batch_size}): {e}")
        continue

    # Ajout dans la collection
    collection.add(
        ids=ids,
        documents=texts,
        metadatas=metadatas,
        embeddings=embeddings
    )

print("Tous les chunks ajoutés (ou ignorés si déjà présents).")


Collection 'fake_news_chunks' récupérée.
145486 chunks chargés depuis data/processed/articles_chunks.csv
0 chunks déjà présents dans la collection.


Ajout chunks: 100%|██████████| 2910/2910 [1:07:25<00:00,  1.39s/it]   


✅ Tous les chunks ajoutés (ou ignorés si déjà présents).
Exemple de document stocké : s...


ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got [[array([-1.81416776e-02, -1.66259743e-02,  2.08692756e-02, -1.75619125e-02,
        1.80299189e-02,  2.60097906e-02,  5.11432113e-03,  1.14210762e-01,
       -2.04584561e-02,  5.41193597e-02, -2.44015623e-02, -1.26532063e-01,
        7.31085055e-03, -1.38087375e-02, -8.85488391e-02, -5.60243055e-02,
       -4.70394045e-02,  1.15937643e-01, -1.96584035e-02,  2.52075624e-02,
        1.93452239e-02, -5.84553555e-02, -2.57454417e-03, -1.32696088e-02,
       -3.99976000e-02, -1.97132248e-02, -7.61035038e-03, -3.05432659e-02,
        1.28386142e-02, -3.06564849e-02,  9.19936523e-02,  2.72286125e-02,
        9.28371400e-02, -2.16248930e-02,  4.68153805e-02, -1.35617855e-03,
        3.60254794e-02, -4.95285392e-02,  1.55769046e-02,  5.89962769e-03,
       -7.28993416e-02, -2.07973160e-02, -7.61383250e-02, -5.29609285e-02,
        2.79479902e-02,  2.00565234e-02,  4.91

In [106]:
# 7 # Fonction RAG pour verdict True/Fake avec Ollama
def rag_verdict_batch(user_queries):
    import ollama
    """
    Fonction qui prend une ou plusieurs déclarations (user_queries)
    et retourne un verdict True/False généré par le modèle Ollama.

    Étapes :
    1. Convertir la requête en liste si besoin.
    2. Initialiser une liste pour stocker les verdicts.
    3. Pour chaque déclaration :
    a) Créer un prompt pour le modèle.
    b) Générer la réponse via Ollama.
    c) Extraire uniquement le texte de la réponse, selon la structure.
    d) Nettoyer le texte pour ne récupérer que True/False.
    e) Gérer les exceptions et erreurs.
    """
# a/ S'assurer que l'entrée est une liste
    if isinstance(user_queries, str):
        user_queries = [user_queries]
# b/ Initialisation de la liste des verdicts
    verdicts = []
# c/  Boucle sur chaque déclaration utilisateur
    for query in user_queries:
        #) Création du prompt pour le modèle
        prompt = f"""
        You are a fact-checker AI.
        Check the following statement and answer only 'True' or 'False'.
        Statement: {query}
        """

        try:
            # Appel à l'API Ollama pour générer le verdict
            response = ollama.generate(model="phi3", prompt=prompt)
            
            # Extraire uniquement le texte de la réponse
            text = ""
            if hasattr(response, "response"):
                text = response.response.strip()
            elif hasattr(response, "message") and hasattr(response.message, "content"):
                text = response.message.content.strip()
            elif hasattr(response, "outputText"):
                text = response.outputText.strip()
            else:
                text = str(response).strip()

            # Nettoyer le texte pour récupérer seulement True/False
            if text.lower().startswith("true"):
                verdicts.append("True")
            elif text.lower().startswith("false"):
                verdicts.append("False")
            else:
                verdicts.append(text)  # Si réponse ambiguë, on garde le texte complet

        except Exception as e:
            #Gestion d'erreur : on ajoute "Erreur" au verdicts
            print(f"Erreur lors de la génération de la réponse : {e}")
            verdicts.append("Erreur")
    # Retourne la liste des verdicts pour toutes les déclarations
    return verdicts



In [ ]:
# 8️ Fonction RAG complète avec contexte + justification
import ollama

def rag_verdict_with_context(user_query, k=5):
    """
    Compare la déclaration utilisateur aux articles similaires dans ChromaDB
    et génère un verdict (TRUE/FAKE) + une justification claire.
    """

    # 1️ Recherche des chunks similaires dans la collection
    results = collection.query(
        query_texts=[user_query],
        n_results=k
    )
    print(type(results))
    print (results)
    # pour voire le type de results
    # 2️ Construction du contexte à partir des chunks récupérés avec label
    context_chunks = []
    metadatas = results["metadatas"][0]  # liste des dicts contenant 'label', 'title', etc.
    documents = results["documents"][0]

    for doc, meta in zip(documents, metadatas):
        label = meta.get("label", "Unknown")
        title = meta.get("title", "")
        # On ajoute label + titre + texte
        context_chunks.append(f"[{label}] {title}\n{doc}")

    context_text = "\n\n".join(context_chunks)  # séparation visuelle
    # # 2️ Construction du contexte à partir des chunks récupérés
    # context_chunks = [doc for doc in results["documents"][0]]
    # context_text = "\n\n".join(context_chunks)  # séparation visuelle entre les chunks

    # 3️ Création du prompt pour le modèle
    prompt = f"""
    You are a fact-checker AI.
    Using the following context from news articles, analyze the user's text and determine if it is TRUE or FAKE.
    Provide a verdict (TRUE or FAKE) and a short explanation.

    User Text:
    {user_query}

    Context from similar articles:
    {context_text}
    """

    # 4️ Génération de la réponse via Ollama (modèle local léger)
    response = ollama.generate(model="phi3:mini", prompt=prompt)

    # 5️ Extraction propre du texte de réponse
    if isinstance(response, dict) and "response" in response:
        text = response["response"].strip()
    elif hasattr(response, "response"):
        text = response.response.strip()
    else:
        text = str(response).strip()

    # 6️ Affichage propre du verdict
    print("=== Résultat RAG ===")
    print(text)
    print("====================\n")

    # 7️ Retour du texte : contient verdict + justification
    return text


# Exemple de test complet
user_query_en = "president donald trump on monday say he plan to nominate liberty university school of law"
verdict_context = rag_verdict_with_context(user_query_en, k=5)

<class 'dict'>
{'ids': [['25638_3', '5751_0', '96_0', '32298_0', '5751_1']], 'embeddings': None, 'documents': [['a time we can all get behind this and say enough is enough we do not support our president in his endorsement of trump and we want the world to know because he s giving liberty university a bad name wahl told the post it makes it seem like we re about populist politics when we re about the gospel of jesus christ this is about gaining the integrity of our school this is an effort to say liberty is not trump university falwell thinks trump is a changed man the problem with his mindset is that trump has not ch', 'washington reuters evangelical christian leader jerry falwell jr will head an education reform task force under u s president donald trump and is keen to cut university regulations including rules on dealing with campus sexual assault the school he heads said falwell the son of the late televangelist jerry falwell sr was described by trump as one of the most respected 

### Evaluation system RAG

In [ ]:
# Évaluation RAG

import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
#from rag_system_chaima import eval_examples

# Importer les fonctions depuis ton module local
from rag_system_chaima.rag_pipeline import rag_pipeline
from rag_system_chaima.chroma_retrieval import get_collection

CSV_FILE = "../rag_system_chaima/eval_examples.csv"


# Charger les exemples
df = pd.read_csv(CSV_FILE)

y_true = []
y_pred = []

print("=== Évaluation RAG sur échantillon CSV ===")

for idx, row in df.iterrows():
    text = row["text"]
    label = row["label"]

    print(f"\nTexte : {text[:80]}...")  # Affiche un extrait
    print(f"Label réel : {label}")

    # Appel du pipeline RAG
    verdict_str = rag_pipeline(text)  # retourne "Verdict: TRUE\nExplanation: ..."

    # Extraire le verdict
    verdict_line = [line for line in verdict_str.splitlines() if line.upper().startswith("VERDICT:")]
    verdict_clean = verdict_line[0].split(":", 1)[1].strip().upper() 

    # Normaliser les verdicts
    if verdict_clean == "FALSE":
        verdict_clean = "FAKE"
    elif verdict_clean == "TRUE":
        verdict_clean = "TRUE"

    print(f"Verdict RAG : {verdict_clean}")

    y_true.append(label.upper())
    y_pred.append(verdict_clean)

# Calcul des métriques
print("\n=== Résultats métriques ===")
print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1-score (macro):", f1_score(y_true, y_pred, average="macro"))
print("\nRapport complet :\n", classification_report(y_true, y_pred, digits=3))


=== Évaluation RAG sur échantillon CSV ===

Texte : donald trump just couldn t wish all americans a happy new year and leave it at t...
Label réel : Fake
Collection 'fake_news_chunks' récupérée.
=== Résultat RAG ===
Verdict: UNKNOWN
Explanation: The context does not support this statement as President Trump has frequently made New Year's Eve toast during his presidency. This claim is inconsistent with the typical behavior of U.S. Presidents and lacks credible source confirmation, suggesting it may be misinformation or satire without factual basis in reality news reporting.

Verdict RAG : UNKNOWN

Texte : year . 2018 will be a great year for america ! ... who use the word hater in a n...
Label réel : Fake
Collection 'fake_news_chunks' récupérée.
=== Résultat RAG ===
Verdict: FAKE
Explanation: The context does not provide any evidence of people using 'hater' in New Year's wishes, which contradicts typical expressions used on such occasions.

Verdict RAG : FAKE

Texte : alan sandoval alan

/home/chaima/Fake-News-Detect/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/chaima/Fake-News-Detect/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/chaima/Fake-News-Detect/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", res